In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import numpy as np


from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
import pandas as pd

#https://machinelearningmastery.com/columntransformer-for-numerical-and-categorical-data/

In [2]:
df= pd.read_csv('train.csv', index_col=0, parse_dates=True)

In [3]:
df['weekday']=df.index.day_of_week #good predictor probably (recurring things happening per day of week)
df['time']=df.index.hour

In [4]:
#df for cas
df_cas = df.drop(['count', 'registered', 'atemp'], axis=1) #also drop atemp, multico 
df_cas

,season,holiday,workingday,weather,temp,humidity,windspeed,casual,weekday,time
datetime,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,81,0.0000,3,5,0
2011-01-01 01:00:00,1,0,0,1,9.02,80,0.0000,8,5,1
2011-01-01 02:00:00,1,0,0,1,9.02,80,0.0000,5,5,2
2011-01-01 03:00:00,1,0,0,1,9.84,75,0.0000,3,5,3
2011-01-01 04:00:00,1,0,0,1,9.84,75,0.0000,0,5,4
...,...,...,...,...,...,...,...,...,...,...
2012-12-19 19:00:00,4,0,1,1,15.58,50,26.0027,7,2,19
2012-12-19 20:00:00,4,0,1,1,14.76,57,15.0013,10,2,20
2012-12-19 21:00:00,4,0,1,1,13.94,61,15.0013,4,2,21


In [5]:
#df for reg
#df_reg = df.drop(['count', 'casual', 'atemp'], axis=1) #also drop atemp, multico 

In [6]:
#look at data types to see what kind of feature engineering to do 
#df_cas.dtypes

In [7]:
#interaction features here? 





In [8]:
X =df_cas[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed', 'weekday', 'time']]
y= df_cas['casual']
X

,season,holiday,workingday,weather,temp,humidity,windspeed,weekday,time
datetime,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,81,0.0000,5,0
2011-01-01 01:00:00,1,0,0,1,9.02,80,0.0000,5,1
2011-01-01 02:00:00,1,0,0,1,9.02,80,0.0000,5,2
2011-01-01 03:00:00,1,0,0,1,9.84,75,0.0000,5,3
2011-01-01 04:00:00,1,0,0,1,9.84,75,0.0000,5,4
...,...,...,...,...,...,...,...,...,...
2012-12-19 19:00:00,4,0,1,1,15.58,50,26.0027,2,19
2012-12-19 20:00:00,4,0,1,1,14.76,57,15.0013,2,20
2012-12-19 21:00:00,4,0,1,1,13.94,61,15.0013,2,21


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
#test, train, split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 10)
X_train.shape, X_test.shape

((8164, 9), (2722, 9))

In [10]:
X_train

,season,holiday,workingday,weather,temp,humidity,windspeed,weekday,time
datetime,,,,,,,,,
2012-10-12 09:00:00,4,0,1,1,20.50,39,19.9995,4,9
2011-09-09 23:00:00,3,0,1,1,25.42,94,6.0032,4,23
2011-06-08 18:00:00,2,0,1,1,36.90,37,15.0013,2,18
2011-10-06 01:00:00,4,0,1,1,19.68,63,12.9980,3,1
2011-10-10 12:00:00,4,1,0,2,28.70,48,6.0032,0,12
...,...,...,...,...,...,...,...,...,...
2012-09-13 21:00:00,3,0,1,1,25.42,65,7.0015,3,21
2012-05-03 04:00:00,2,0,1,2,18.86,88,16.9979,3,4
2011-04-01 21:00:00,2,0,1,1,13.12,53,8.9981,4,21


In [11]:
#interaction features: working day and all continuous
#(will make partially categorical feature)-> doing before coltrans (otherwise too complicated)



#pipeline to bin, and OHE for final

In [12]:
#custom func: bin wind into two (32 divide) to make more predictive (based on dist) -> see EDA for custom bin
def custom_bin(df):
    
    for a in df:
        df.loc[ (df['windspeed']> 0) & (df['windspeed'] <= 25), 'windspeed'] = 0
        df.loc[ (df['windspeed']> 25), 'windspeed'] = 1
    
        return df[['windspeed']]
#for in pipe: ('cust_bin', FunctionTransformer(custom_bin), ['windspeed'])

In [13]:
 #wind (custom bins and then encode)
pipe1 = Pipeline([
    ('custom_bin', FunctionTransformer(custom_bin)), ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

In [14]:
from sklearn.preprocessing import PolynomialFeatures

In [15]:
pipe2 = Pipeline([ ('scaler', StandardScaler()), ('poly_eng', PolynomialFeatures(degree=2))
])

#temp

In [16]:
#humidity (& interaction terms??)

pipe3 = Pipeline([('scaler', StandardScaler())])


In [17]:
# feature engineering for categorical
pipe4 = Pipeline([('ohe', OneHotEncoder(handle_unknown="ignore"))])

In [18]:
trans = ColumnTransformer([
    ('pipe1', pipe1, ['windspeed']), 
    ('pipe2', pipe2, ['temp']), 
    ('pipe3', pipe3, ['humidity']), 
    ('pipe4', pipe4,['weather', 'workingday', 'holiday', 'season', 'weekday', 'time'])

]) #1D scalar mistake, but if try to pass col as a list of 1 item, pipes are repeated and coltrans doesnt work, PDP instead? 

In [19]:
#transform to see tangible output (comment out later)
#X_train_trans = trans.fit_transform(X_train)
#X_test_trans = transform(X_test)

In [20]:
#X_train_trans

In [21]:
#to see a tangible output
#X_train_look = pd.DataFrame(X_train_trans.todense())
#X_train_look #sparse- why negative values?? and so many rows? 

In [22]:
from sklearn.linear_model import LinearRegression

In [23]:
model0 =LinearRegression()

In [27]:
pipe5 = Pipeline(steps=[
    ('trans', trans), 
    ('model0', LinearRegression())
     ])

In [28]:
pipe5.fit(X_train, y_train)
# make predictions

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('pipe1',
                                                  Pipeline(steps=[('custom_bin',
                                                                   FunctionTransformer(func=<function custom_bin at 0x000002067D44D940>)),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['windspeed']),
                                                 ('pipe2',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('poly_eng',
                                                                   PolynomialFeatures())]),
                                                  ['

In [29]:
# make predictions
yhat = pipe5.predict(X_test)

In [30]:
from sklearn import metrics

In [31]:
r2_test = metrics.r2_score(y_test, yhat)

r2_test
#72 without interaction features (57)

0.5669762251370634

In [ ]:
#grid search to see which parameters important 




In [ ]:
#print("model score: %.3f" % pipeline.score(X_test, y_test))
# get predictions from the pipeline
#print(pipeline.predict(X_test))

# get prediction probabilities from the pipeline 
#print(pipeline.predict_proba(X_test)[:, 1])

In [21]:
#linreg without FE
#m = LinearRegression()
#m.fit(X_train,y_train)
#m.score(X_test, y_test)
# m.predict(X)
#y_pred = m.predict(X)
#df['y_pred'] = y_pred